In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from prophet import Prophet
import ipywidgets as widgets
from IPython.display import display, clear_output

# ====== CARGA DE DATOS ======
df = pd.read_excel('calidad_agua.xlsx', parse_dates=['Fecha'])
df = df.sort_values('Fecha').reset_index(drop=True)

# ====== WIDGETS ======
start_picker = widgets.DatePicker(description='Desde', value=df['Fecha'].min())
end_picker   = widgets.DatePicker(description='Hasta', value=df['Fecha'].max())

vars_disp = ['pH', 'Oxígeno disuelto (mg/L)', 'Turbidez (NTU)', 'Temperatura (°C)', 'Nitratos (mg/L)']
multi_select = widgets.SelectMultiple(options=vars_disp, description='Variables', value=tuple(vars_disp))

proj_dropdown = widgets.Dropdown(options=vars_disp, description='Proyección:')
proj_slider = widgets.IntSlider(description='Meses futuro', min=1, max=24, value=12)

btn = widgets.Button(description='Actualizar dashboard', button_style='success')

# ====== FUNCIÓN PRINCIPAL ======
def on_button_click(b):
    clear_output(wait=True)

    # FILTRAR
    start, end = start_picker.value, end_picker.value
    mask = (df['Fecha'] >= pd.to_datetime(start)) & (df['Fecha'] <= pd.to_datetime(end))
    sub = df.loc[mask].copy()

    if sub.empty:
        print("❗ No hay datos en ese rango.")
        return

    # GRÁFICAS DE LÍNEA
    for var in multi_select.value:
        fig = px.line(sub, x='Fecha', y=var, title=f"{var} en el tiempo")
        fig.update_traces(mode='lines+markers')
        fig.show()

    # HISTOGRAMAS
    for var in multi_select.value:
        fig = px.histogram(sub, x=var, nbins=20, title=f"Distribución de {var}")
        fig.show()

    # BOXPLOTS
    for var in multi_select.value:
        fig = px.box(sub, y=var, title=f"Boxplot de {var}")
        fig.show()

    # HEATMAP DE CORRELACIÓN
    corr = sub[vars_disp].corr()
    fig = px.imshow(corr, text_auto=True, title="Matriz de correlación")
    fig.show()

    # PROYECCIÓN CON PROPHET
    var = proj_dropdown.value
    df_prophet = sub[['Fecha', var]].rename(columns={'Fecha': 'ds', var: 'y'})

    m = Prophet()
    m.fit(df_prophet)

    future = m.make_future_dataframe(periods=proj_slider.value, freq='MS')
    forecast = m.predict(future)

    fig_forecast = go.Figure()
    fig_forecast.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicción'))
    fig_forecast.add_trace(go.Scatter(x=df_prophet['ds'], y=df_prophet['y'], name='Histórico'))
    fig_forecast.update_layout(title=f"Proyección realista de {var} con Prophet", xaxis_title='Fecha', yaxis_title=var)
    fig_forecast.show()

    # 3 GRÁFICAS GLOBALES SIN FILTRO

    # 1) Líneas superpuestas
    fig_all = go.Figure()
    for var in vars_disp:
        fig_all.add_trace(go.Scatter(x=df['Fecha'], y=df[var], mode='lines+markers', name=var))
    fig_all.update_layout(title="Todas las variables (líneas superpuestas)", xaxis_title="Fecha")
    fig_all.show()

    # 2) Área apilada
    fig_area = go.Figure()
    for var in vars_disp:
        fig_area.add_trace(go.Scatter(
            x=df['Fecha'], y=df[var],
            stackgroup='one',
            mode='none',
            name=var
        ))
    fig_area.update_layout(title="Evolución acumulada (área apilada)", xaxis_title="Fecha")
    fig_area.show()

    # 3) Boxplots juntos
    fig_box = go.Figure()
    for var in vars_disp:
        fig_box.add_trace(go.Box(y=df[var], name=var))
    fig_box.update_layout(title="Comparación de rangos y outliers (boxplots)")
    fig_box.show()

    # Mostrar UI
    display(ui)

# ====== UI ======
btn.on_click(on_button_click)
ui = widgets.VBox([
    widgets.HBox([start_picker, end_picker]),
    widgets.HBox([multi_select, proj_dropdown, proj_slider]),
    btn
])
display(ui)


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpl2bcpc34/r6ipf881.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpl2bcpc34/l1sdnu_c.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=59640', 'data', 'file=/tmp/tmpl2bcpc34/r6ipf881.json', 'init=/tmp/tmpl2bcpc34/l1sdnu_c.json', 'output', 'file=/tmp/tmpl2bcpc34/prophet_model88vuiizt/prophet_model-20250610135837.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:58:37 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:58:37 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
